In [1]:
import numpy as np
import re
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer

In [2]:
stop_words = ['i', 'me', 'my', 'myself', 'we', 
              'our', 'ours', 'ourselves', 'you', 
              "you're", "you've", "you'll", "you'd", 
              'your', 'yours', 'yourself', 'yourselves', 
              'he', 'him', 'his', 'himself', 'she', "she's", 
              'her', 'hers', 'herself', 'it', "it's", 'its', 
              'itself', 'they', 'them', 'their', 'theirs', 'themselves', 
              'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 
              'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 
              'been', 'being', 'have', 'has', 'had', 'having', 'do', 
              'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 
              'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 
              'by', 'for', 'with', 'about', 'against', 'between', 'into',
              'through', 'during', 'before', 'after', 'above', 'below', 
              'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 
              'under', 'again', 'further', 'then', 'once', 'here', 'there', 
              'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each',
              'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor',
              'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 
              's', 't', 'can', 'will', 'just', 'don', "don't", 'should', 
              "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 
              'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', 
              "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't",
              'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 
              'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
              "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [3]:
news_data = pd.read_csv('../../data/aulas/news.csv')
news_data.head()

,id,title,author,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,1


In [4]:
news_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   label   20800 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 487.6+ KB


In [5]:
# Substitui valores nulos (NaN) no DataFrame news_data por strings vazias (''), garantindo que não haja valores ausentes nas operações subsequentes.
news_data = news_data.fillna('')

In [6]:
# Conta e exibe o número de valores nulos (ausentes) em cada coluna do DataFrame news_data, para verificar se ainda existem valores faltantes após o preenchimento.
news_data.isnull().sum()

id        0
title     0
author    0
label     0
dtype: int64

In [7]:
# Substitui valores nulos (NaN) no DataFrame news_data por strings vazias (''), garantindo que não haja valores ausentes nas operações subsequentes.
news_data = news_data.fillna('')

# Stemming

In [8]:
# Cria uma instância do PorterStemmer, um algoritmo usado para realizar stemming, que reduz as palavras às suas raízes ou radicais (ex.: "running" → "run").
port_stem = PorterStemmer()

In [9]:
# Define a função stemming para processar e limpar texto, reduzindo palavras ao seu radical.
# Parâmetros:
#   - content: texto a ser processado.
# Passos:
#   1. Usa re.sub para substituir todos os caracteres que não sejam letras (a-z, A-Z) por espaços.
#   2. Converte o texto para letras minúsculas com lower().
#   3. Divide o texto em uma lista de palavras usando split().
#   4. Aplica o stemming a cada palavra na lista, ignorando palavras presentes em stop_words.
#      - Usa a função stem do PorterStemmer para reduzir palavras ao radical.
#   5. Junta as palavras processadas de volta em uma string usando ' '.join().
# Retorno:
#   - Uma string contendo o texto limpo e processado.

def stemming(content):
    review = re.sub('[^a-zA-Z]',' ',content)
    review = review.lower()
    review = review.split()
    review = [port_stem.stem(word) for word in review if not word in stop_words]
    review = ' '.join(review)
    return review

In [10]:
# Aplica a função stemming a cada elemento da coluna 'content' no DataFrame news_data, processando e limpando o texto em todas as linhas. O resultado é armazenado de volta na coluna 'content'.
news_data['content'] = news_data['content'].apply(stemming)

In [11]:
# Exibe os primeiros 5 registros da coluna 'content' no DataFrame news_data, permitindo visualizar o conteúdo processado após a aplicação do stemming.
news_data['content'].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [12]:
# X: Extrai os valores da coluna 'content' como um array NumPy, representando as variáveis independentes (features) para o modelo.
# y: Extrai os valores da coluna 'label' como um array NumPy, representando as variáveis dependentes (alvos) para o modelo

X = news_data['content'].values
y = news_data['label'].values

# TF-IDF

In [13]:
# Importa a classe TfidfVectorizer da biblioteca scikit-learn, usada para converter texto em uma matriz de recursos numéricos baseada na frequência do termo (TF) e frequência inversa do documento (IDF).
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
# Cria uma instância do TfidfVectorizer para transformar o texto em uma representação numérica baseada no TF-IDF.
# fit_transform(X): Ajusta o vetorizador aos dados de texto X e, ao mesmo tempo, transforma-os em uma matriz esparsa
# onde cada linha representa um documento e cada coluna representa uma palavra única no vocabulário.

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

In [17]:
# Importa a função train_test_split da biblioteca scikit-learn, usada para dividir os dados em conjuntos de treino e teste.
from sklearn.model_selection import train_test_split

In [18]:
# Divide os dados em conjuntos de treino e teste:
# X_train e y_train são os dados de treino (variáveis independentes e dependentes, respectivamente).
# X_test e y_test são os dados de teste (variáveis independentes e dependentes, respectivamente).
# test_size=0.3 significa que 30% dos dados serão usados para teste, e o restante (70%) será usado para treino.
# random_state=101 garante que a divisão dos dados seja reprodutível.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [19]:
# Exibe o conteúdo de X_train, que é a matriz de características (features) do conjunto de treino.
# Isso permite visualizar as primeiras entradas dos dados de treino após a transformação TF-IDF.
print(X_train)

  (0, 12958)	0.33950622872556196
  (0, 12842)	0.3232304689577807
  (0, 5469)	0.30140922161045747
  (0, 2220)	0.31004399946914174
  (0, 1268)	0.33331847204135595
  (0, 8623)	0.30846964228664014
  (0, 1627)	0.2729433574074474
  (0, 311)	0.2370646474139764
  (0, 8534)	0.2855674224224232
  (0, 10793)	0.36114206713970187
  (0, 15295)	0.08655594427821867
  (0, 16996)	0.08821502722057027
  (0, 10306)	0.17054081261709328
  (1, 3732)	0.3915966363846314
  (1, 3314)	0.3862190396394902
  (1, 10677)	0.4527188282827226
  (1, 11259)	0.3145342713467437
  (1, 12840)	0.3096081376574435
  (1, 13014)	0.33777512378155045
  (1, 469)	0.2845240719071562
  (1, 11155)	0.32046466547016283
  (2, 8695)	0.5453031510728447
  (2, 15295)	0.3851115774131578
  (2, 16996)	0.3924933009252318
  (2, 10306)	0.37939185984259366
  :	:
  (14557, 4680)	0.25775069892089064
  (14557, 13775)	0.2723897360085363
  (14557, 5233)	0.2556282697087412
  (14558, 8209)	0.407793149055482
  (14558, 8061)	0.3607746576291789
  (14558, 14787)	0.

In [20]:
# Importa a classe MultinomialNB da biblioteca scikit-learn, que implementa o classificador Naive Bayes para problemas de classificação,
# especialmente adequado para dados de texto, onde as variáveis seguem uma distribuição multinomial.

from sklearn.naive_bayes import MultinomialNB

In [21]:
# Cria uma instância do classificador MultinomialNB, que será utilizado para treinar o modelo Naive Bayes
# com base nos dados de treino, adequado para problemas de classificação com características distribuídas de forma multinomial.

nb = MultinomialNB()

In [22]:
# Treina o modelo Naive Bayes (MultinomialNB) usando os dados de treino (X_train e y_train),
# ajustando os parâmetros do modelo para aprender a relação entre as características e os rótulos.

nb.fit(X_train, y_train)

MultinomialNB()

In [23]:
# Faz previsões sobre o conjunto de teste (X_test) usando o modelo Naive Bayes treinado (nb),
# gerando os rótulos preditos para as amostras no conjunto de teste.

preds = nb.predict(X_test)

In [24]:
preds

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [25]:
y_test

array([0, 1, 0, ..., 0, 1, 1], dtype=int64)

In [26]:
# Importa a função accuracy_score da biblioteca scikit-learn, que é usada para calcular a precisão do modelo,
# comparando os rótulos preditos com os rótulos reais (verdadeiros) do conjunto de teste.

from sklearn.metrics import accuracy_score

In [27]:
# Calcula e exibe a precisão do modelo, comparando os rótulos reais (y_test) com os rótulos preditos (preds),
# mostrando a porcentagem de acertos do modelo no conjunto de teste.

print(accuracy_score(y_test, preds))

0.9496794871794871
